# RD sim on metaballs (in silico hybridization)
### white spot (20) x black spot (20) = 400 ISHs

In [ ]:
%load_ext autoreload
%autoreload 1

import os
import itertools

from joblib import Parallel, delayed
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

from rdmb import *
from quant import *
%aimport rdmb, quant

%matplotlib inline

### Parents selection (white-spotted and black-spotted)

In [ ]:
df_uni = pd.read_csv("uni.csv", index_col=0)
df_p = df_uni[(df_uni.lightness > 0.0) & (df_uni.lightness < 1.0)]

df_p.reset_index(drop=True).to_csv("uni_p.csv")

df_p.head()

In [ ]:
fig = plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.scatter(x=df_p['C'], y=df_p['A'], color="skyblue")
plt.ylim(0.065, 0.145)
plt.xlabel("param C")
plt.ylabel("param A")
plt.subplot(1, 2, 2)
plt.imshow(Image.open("rd2d_mat_AC_A0.14-0.07_C-0.1-0.4.jpg"), cmap="gray")
plt.axis('off')
plt.show()

In [ ]:
df_p_ws = df_p[(df_p.complexity < 0.4)
               & (df_p.lightness < 0.4)
               & (df_p.lightness > 0.03)]
df_p_bs = df_p[(df_p.complexity < 0.4)
               & (df_p.lightness > 0.6)
               & (df_p.lightness < 0.97)]
print("Number of candidates for")
print("    white-spotted parents:", len(df_p_ws))
print("    black-spotted parents:", len(df_p_bs))

In [ ]:
fig = plt.figure(figsize=(6, 6))
plt.ylim(0.065, 0.145)
plt.scatter(x=df_p['C'], y=df_p['A'], color="skyblue")
plt.scatter(x=df_p_ws['C'], y=df_p_ws['A'], color="orange")
plt.scatter(x=df_p_bs['C'], y=df_p_bs['A'], color="green")
plt.show()

In [ ]:
np.random.seed(42)

# select 20 parents each
df_p_ws20 = df_p_ws.sample(20)
df_p_bs20 = df_p_bs.sample(20)

# subsets for figure
df_p_ws10 = df_p_ws20.sample(10)
df_p_bs10 = df_p_bs20.sample(10)

In [ ]:
fig = plt.figure(figsize=(6, 6))
plt.ylim(0.065, 0.145)

plt.scatter(x=df_p['C'], y=df_p['A'], color="skyblue")

plt.scatter(x=df_p_ws20['C'], y=df_p_ws20['A'], s=120, color="orange", alpha=0.2)
plt.scatter(x=df_p_bs20['C'], y=df_p_bs20['A'], s=120, color="green", alpha=0.2)

plt.scatter(x=df_p_ws10['C'], y=df_p_ws10['A'], color="orange")
plt.scatter(x=df_p_bs10['C'], y=df_p_bs10['A'], color="green")

plt.show()

### RD calculation for in silico hybrids

In [ ]:
def run_rdmb_grad_AC(fnbase, out_dir, ACs):
    
    mb_obj_file = "mbobj/mb_hq.obj.gz"

    As = ACs[0]
    Cs = ACs[1]

    out_file_base = (out_dir + "/" + fnbase
                     + "_hq_A{:.4f}_C{:.3f}-A{:.4f}_C{:.3f}"
                     .format(As[0], Cs[0], As[1], Cs[1]))
    
    rdmb.rdmb_grad_AC(mb_obj_file,
                      out_file_base,
                      px0=-1.6, px1=1.6,
                      pa0=As[0], pa1=As[1],
                      pc0=Cs[0], pc1=Cs[1],
                      max_ite=2000)

    return out_file_base

In [ ]:
%%time

fnbase = "ws20-bs20"
out_dir = "rd_out/ish"

try:
    os.makedirs("./"+out_dir)
except FileExistsError:
    pass

np.random.seed(42)

df_p_ws20_sort = df_p_ws20.sort_values('A', ascending=False)
df_p_bs20_sort = df_p_bs20.sort_values('A', ascending=False)

wsAs = df_p_ws20_sort.A.values
wsCs = df_p_ws20_sort.C.values

bsAs = df_p_bs20_sort.A.values
bsCs = df_p_bs20_sort.C.values

Ass = itertools.product(wsAs, bsAs)
Css = itertools.product(wsCs, bsCs)

ish_fnbases = Parallel(n_jobs=36, verbose=1, max_nbytes=None)(
                    [delayed(run_rdmb_grad_AC)(
                        fnbase=fnbase,
                        out_dir=out_dir,
                        ACs=ACs)
                     for ACs in zip(Ass, Css)])

!slackpost.sh -m "ish_rdmb おわったで～"

### Rendering for quantification

In [ ]:
pngfnames_q = Parallel(n_jobs=36, verbose=1, max_nbytes=None)(
                    [delayed(rdmb.rdmb_povray_q)(ish_fnbase,
                                                 time_point=2000,
                                                 width=400,
                                                 height=200,
                                                 angle=10)
                     for ish_fnbase in ish_fnbases])

In [ ]:
display(Image.open(pngfnames_q[197]))

### Quantification

In [ ]:
imgs = [Image.open(pngfname_q) for pngfname_q in pngfnames_q]
imgs_crop_scale = [quant.crop_center(img, 128, 128)
                   .convert('L')
                   .resize((420, 420), resample=Image.LANCZOS)
                   for img in imgs]
imgs_mono = [quant.binarize(np.asarray(img))
             for img in imgs_crop_scale]

Ls = np.array([quant.quant(img)[0] for img in imgs_mono])
PCSs = np.array([quant.quant(img)[1] for img in imgs_mono])

df_ish = pd.DataFrame(np.c_[ish_fnbases,
                            pngfnames_q,
                            Ls,
                            PCSs],
                      columns=["file_base",
                               "png_file_q",
                               "lightness",
                               "complexity"])
df_ish.to_csv("ish.csv")

In [ ]:
df_ish.head()

### Rendering for visualization

In [ ]:
pngfnames_col = Parallel(n_jobs=20, verbose=1, max_nbytes=None)(
                    [delayed(rdmb_povray_color)(ish_fnbase,
                                                time_point=2000,
                                                width=300,
                                                height=300,
                                                rotx=0, roty=0, rotz=45,
                                                angle=8,
                                                mode="AC")
                     for ish_fnbase in ish_fnbases])

In [ ]:
display(Image.open(pngfnames_col[197]))

In [ ]:
ws20_pngfnames = [file_base+"_color_02000.png"
                  for file_base in df_p_ws20_sort.file_base.values]
bs20_pngfnames = [file_base+"_color_02000.png"
                  for file_base in df_p_bs20_sort.file_base.values]

In [ ]:
ijmax=20

img = Image.new('RGBA', (280*ijmax+320, 280*ijmax+320), color='white')

for i in range(ijmax):
    im = Image.open(ws20_pngfnames[i])
    img.paste(im, (0, 120+280*(i+1)), mask=im)
    im = Image.open(bs20_pngfnames[i])
    img.paste(im, (130+280*(i+1), 0), mask=im)
    
    for j in range(ijmax):
        im = Image.open(pngfnames_col[i*ijmax+j])
        img.paste(im, (280*j+260, 280*i+260), mask=im)
        
img.convert(mode="RGB").save("ish_20x20.jpg", quality=95)
display(img.resize((800, 800), Image.LANCZOS))

### subset (10x10)

In [ ]:
ws10s, = np.where(np.isin(df_p_ws20_sort.index.values,
                          df_p_ws10.index.values))
bs10s, = np.where(np.isin(df_p_bs20_sort.index.values,
                          df_p_bs10.index.values))
print(ws10s)
print(bs10s)

In [ ]:
img = Image.new('RGBA', (280*10+320, 280*10+320))

for i, wi in enumerate(ws10s):
    im = Image.open(ws20_pngfnames[wi])
    img.paste(im, (0, 120+280*(i+1)), mask=im)
for j, bj in enumerate(bs10s):
    im = Image.open(bs20_pngfnames[bj])
    img.paste(im, (130+280*(j+1), 0), mask=im)
    
for i, wi in enumerate(ws10s):
    for j, bj in enumerate(bs10s):
        im = Image.open(pngfnames_col[wi*20+bj])
        img.paste(im, (280*j+260, 280*i+260), mask=im)
        
img.convert(mode="RGBA").save("ish_10x10.png")

display(img.resize((800, 800), Image.LANCZOS))

In [ ]:
import seaborn as sns

sns.set_context('talk')

x, y = np.meshgrid(
    np.linspace(0, 1, 100),
    np.linspace(0, 1, 100),
)

A = np.linspace(0.07, 0.12, num=100)
C = np.linspace(-0.1, 0.31, num=100)

nA = (A-0.07)/(0.12-0.07)
nC = (C+0.1)/(0.31+0.1)
    
fig = plt.figure(figsize=(3, 3))

plt.imshow(
    np.dstack((y, np.zeros_like(x), x)),
    extent = (
        np.min(C), np.max(C),
        np.min(A), np.max(A),
    ),
    aspect = 'auto',
    origin = 'lower',
)
plt.xticks([0, 0.1, 0.2, 0.3])
plt.yticks([0.08, 0.10, 0.12])

plt.xlabel('Parameter $\it{C}$', fontsize=20)
plt.ylabel('Parameter $\it{A}$', fontsize=20)

plt.savefig("colorbar_2D.pdf")

plt.show()